In [1]:
import time
import random
import numpy as np
s = time.perf_counter()

In [2]:
class Route():
    def __init__(self):
        self.fitness = 0
        self.board = np.zeros(n)

    def cal_fitness(self):
        self.fitness = 0
        for i in range(n-1):
            for j in range(i+1, n):
                if self.board[i] == self.board[j]+(j-i) or self.board[i] == self.board[j]-(j-i) :
                    self.fitness-=1

class FirstRoute(Route) :
    def __init__(self) :
        self.fitness = 0
        self.board = np.random.permutation(n)
        
        

In [3]:
def best(l):
    ans = l[0]
    for r in l:
        if r.fitness>ans.fitness:
            ans = r
    return ans

def average(l):
    sum = 0
    for r in l:
        sum+=r.fitness
    return sum/(len(l))

def index_of_worst(l):
    ans = 0
    for i in range(len(l)):
        if l[i].fitness<l[ans].fitness:
            ans = i
    return ans


In [4]:
def tournament(Population, k):
        index = random.sample(range(len(Population)), k)
        l = []
        for i in index:
            l.append(Population[i])
        return best(l)

def tournament_selection(Population, m):
    k = random.randint(2,8)                #m times choose k element and return the best of k choosen element(m elements)
    
    newPopulation = []
    for ـ in range(m):
        newPopulation.append(tournament(Population, k))

    return newPopulation

In [5]:
def sorted_union(nestedList):
        newList = []
        for list in nestedList:
            newList += list
        newList.sort()
        return newList
def smallest_index_which_is_not_in_list(list):
    if list :
        for i in range (len(list)+1):
            if i not in list:
                return i
    else:
        return 0
def recombination(groups, parent1, parent2):
    l = [-1 for _ in range(n)]
    mainParent = parent1
    for group in groups:
        for i in group:
            l[i] = mainParent[i]
        mainParent = parent1 if np.array_equal(mainParent, parent2) else parent2
        
    ans = Route()
    ans.board = l
    ans.cal_fitness()
    return ans ;

def get_groups(parent1,parent2):
    lO = []
    while len(sorted_union(lO)) != len(parent1) :
        startPoint = smallest_index_which_is_not_in_list(sorted_union(lO))
        lI = []
        while True:
            lI.append(startPoint)
            city2 = parent2[startPoint]
            for i,city1 in enumerate(parent1):
                if city1 == city2:
                    startPoint = i
                    break
                
            if startPoint in lI :
                lO.append(lI)
                break        

    return recombination(lO, parent1, parent2)

def Cycle_Recombination(parents):
    children = []
    for _ in range(len(parents)):
        i=random.randint(0,len(parents)-1)
        j=random.randint(0,len(parents)-1)
        children.append(get_groups(parents[i].board, parents[j].board))
    return children

In [6]:
def Order_recombination(parents):
    def recombination(parent1, parent2):
        def get_start_end():
            a = random.sample(range(n),2 )
            a.sort()
            return a
        
        l = [-1 for _ in range(n)]
        startPoint, endPoint = get_start_end()
        for i in range(startPoint, endPoint+1):
                l[i]=parent1[i]

        j = endPoint+1 if endPoint != n-1 else 0 

        for i in range(endPoint+1, n):
            while parent2[j] in l:
                j+=1
                if j==n:
                    j = 0
            l[i]=parent2[j]
            j+=1
            if j==n:
                    j = 0

        for i in range(startPoint):
            while parent2[j] in l:
                j+=1
                if j==n:
                    j = 0
            l[i]=parent2[j]
            j+=1
            if j==n:
                    j = 0

        return l
    children = Route()
    for i in range(0,len(parents),2):
        children.board.append(recombination(parents[i].board, parents[i+1].board))
    return children

In [7]:
def mutation(city):
    i = random.randint(0,n-1)
    j = random.randint(0,n-1)
    city.board[i], city.board[j] = city.board[j], city.board[i]
    city.cal_fitness()
    return city

def swap_mutation(population, PMutation):
    for _ in range(int(len(population)*PMutation)):
        i = random.randint(0,len(population)-1)
        population[i] = mutation(population[i])
    return population


In [8]:
def generational_replacement(parents,chidren, PRep):
    for _ in range (int(populationNum*PRep)):
        indexOfWorst = index_of_worst(parents)
        parents[indexOfWorst] = best(chidren)
        chidren.remove(best(chidren))
    return parents

In [9]:
def swap(route,r):
        i = random.randint(0,n-r)
        b = Route()
        b.board = np.copy(route.board)
        b.board[i:i+r] = np.copy(np.flip(route.board[i:i+r]))
        b.cal_fitness()
        return b
    
def get_neighbours(route, r):
    
    neighbours = np.array([Route() for _ in range(n//r)])
    for i in range(n//r):
        neighbours[i] = swap(route,r)

    return neighbours

In [10]:
import math
def tabu_search(selectedNeighbour, siblings, originalBoard):
    siblings.remove(originalBoard)
    for sibling in siblings:
        j = 0
        for i in range(n):
            if sibling.board[i] == selectedNeighbour.board[i]:
                j+=1
            if j>=int((3*n) / 4):
                return originalBoard
    return selectedNeighbour

def simulated_annealing(selectedNeighbour, originalBoard, averageFitness):
    delta = selectedNeighbour.fitness - originalBoard.fitness
    if delta>=0 :
        return selectedNeighbour
    elif math.exp(delta/averageFitness) > random.uniform(0,1):
        return selectedNeighbour
    return originalBoard

def kth_best(boards, k):
        for i in range(k-1):
            j = 0
            for u in range(len(boards)):
                if boards[u].fitness > boards[j].fitness :
                    j = u
            del boards[j]
            
        j = 0
        for i in range(len(boards)):
            if boards[i].fitness > boards[j].fitness :
                j = i
        return boards[j]
    
def local_search(siblings, neighbours, originalBoard):
    k = random.randint(1,6)
    kthBest = kth_best(neighbours,k)
    
    if random.randint(0,1):
        return tabu_search(kthBest, siblings[:], originalBoard)
            
    return simulated_annealing(kthBest, originalBoard, average(siblings))

In [11]:
def lamarcian(selectedNeighbour):
    return selectedNeighbour
def baldwinidan(selectedNeighbour, originalBoard):
    originalBoard.fitness = selectedNeighbour.fitness
    return originalBoard

def memetic(selectedNeighbour, originalBoard) :
    return lamarcian(selectedNeighbour) if random.randint(0,1) else baldwinidan(selectedNeighbour, originalBoard)

In [12]:

def improve(parents, r) :
    for i in range(len(parents)):
        neighbours = list(get_neighbours(parents[i],r))      
        selectedNeighbour = local_search(parents,neighbours, parents[i])
        
        if not np.array_equal(selectedNeighbour.board, parents[i].board):
            parents[i] = memetic(selectedNeighbour, parents[i])
    return parents

In [13]:
def plot(aveIter, bestIter):
    import pandas as pd
    import matplotlib.pyplot as plt
    
    data = pd.DataFrame({"avegrage":aveIter, "best":bestIter})
    # plt.xlabel('generation', fontsize = 15)
    # plt.ylabel('fitness', fontsize = 15)
    data.plot()
    
    print(time.perf_counter()-s)

In [14]:
def show(board):
    import matplotlib.pyplot as plt
    chess = np.zeros((n,n))
    for i in range(n):
        for j in range(n):
            if (j+i)%2 :
                chess[i][j] = -1
    for i in range(n):
        chess[i][board[i]]= 1
    print(chess)
    plt.figure(figsize=(n,n))
    plt.imshow(chess,cmap='gray')
    plt.axis(False)
    plt.show()


In [15]:
def main(n,firstPopulationNum, populationNum,PMutation,PRep):
    firstPopulation = np.array([FirstRoute() for _ in range(firstPopulationNum)])
    for i in range(firstPopulationNum):
        firstPopulation[i] = FirstRoute()
        firstPopulation[i].cal_fitness()
    parents = firstPopulation
    bestAns=best(parents).fitness
    while bestAns !=0 :                
        bestAns=best(parents).fitness
        parents = tournament_selection(parents, populationNum)     
        children = Cycle_Recombination(parents)                    
        children = improve(children, random.randint(2,n//8))
        children = swap_mutation(children,PMutation)        
        # children = improve(children, random.randint(2,n//10))
        parents = generational_replacement(parents, children, PRep) 
        print(bestAns)
        
    print("ans:",bestAns, best(parents).board)
    # plot(aveIter, bestIter)
    return(best(parents))


In [16]:
n = 300
firstPopulationNum = 400
populationNum = 250
PMutation = 0.65    
PRep = 0.1      
ansBoard = main(n,firstPopulationNum, populationNum,PMutation,PRep)
show(ansBoard)

-153
-153
-143
-143
-134
-129
-126
-124
-119
-118
-113
-108
-103
-102
-99
-98
-96
-95
-93
-90
-90
-88
-88
-88
-86
-83
-79
-78
-76
-75
-73
-73
-72
-70
-70
-67
-63
-63
-63
-61
-60
-58
-58
-58
-57
-57
-56
-56
-55
-55
-55
-54
-53
-53
-53
-51
-51
-50
-50
-49
-48
-48
-47
-45
-45
-44
-44
-44
-43
-43
-42
-42
-41
-41
-41
-40
-40
-39
-39
-37
-37
-35
-35
-35
-34
-34
-34
-33
-33
-32
-32
-31
-30
-30
-29
-29
-29
-29
-28
-28
-28
-28
-28
-28
-28
-28
-28
-27
-27
-27
-27
-27
-27
-27
-27
-26
-26
-26
-26
-25
-25
-25
-24
-24
-23
-23
-23
-23
-23
-23
-22
-22
-22
-22
-22
-22
-21
-21
-21
-20
-20
-20
-20
-20
-20
-19
-19
-19
-18
-18
-18
-17
-17
-17
-17
-17
-17
-17
-16
-16
-16
-16
-16
-16
-16
-16
-15
-15
-15
-15
-15
-15
-15
-14
-14
-14
-14
-14
-14
-13
-13
-13
-13
-12
-12
-11
-11
-11
-11
-11
-11
-11
-11
-11
-11
-11
-11
-11
-11
-11
-10
-10
-10
-10
-9
-9
-9
-9
-9
-9
-9
-9
-9
-9
-9
-9
-9
-9
-9
-9
-9
-9
-9
-9
-9
-9
-9
-9
-9
-9
-9
-9
-9
-9
-9
-9
-9
-9
-9
-9
-9
-9
-9
-9
-9
-9
-9
-9
-9
-9
-9
-9
-8
-8
-8
-8
-8
-8
-8
-8
-8

TypeError: 'Route' object is not subscriptable

In [19]:
show(ansBoard.board)

[[ 0. -1.  0. ... -1.  0. -1.]
 [-1.  0. -1. ...  0. -1.  0.]
 [ 0. -1.  0. ... -1.  0. -1.]
 ...
 [-1.  0. -1. ...  0. -1.  0.]
 [ 0. -1.  0. ... -1.  0. -1.]
 [-1.  0. -1. ...  0. -1.  0.]]
